<h1 style="text-align:center">Building an API for Your Model</h1>

- online-ds-ft-0706200
- 11/03/20

#### What is an API?

An API allows you to interact with other computers. It takes a request and based on the information in that request it triggers the computer to run certain tasks and generate a specific output.  

<img src='images/api.png' width='400px'/>

<b>Example</b>

<img style='width:200px' src= 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Hotdog_-_Evan_Swigart.jpg/1200px-Hotdog_-_Evan_Swigart.jpg' />

In [ ]:
USE_CLOUD = False

In [ ]:
import requests
if USE_CLOUD:
    try:
        url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Hotdog_-_Evan_Swigart.jpg/1200px-Hotdog_-_Evan_Swigart.jpg'
        # url = 'http://2.bp.blogspot.com/-kXJaeb6-Zuc/Uz2ZLpl_YeI/AAAAAAAAEFc/fFYzeDAh-SY/s1600/Peanut+Butter+and+Honey+Banana+Dog+002.JPG'
        payload = {"image": url}
        r = requests.post('http://ec2-3-16-164-62.us-east-2.compute.amazonaws.com/predict', data={'image':url}).json()
        display(r)
    except Exception as e:
        print('[!] ERROR: '+str(e))
else:
    print('[!] Skipping .post command to EC2 model since USE_CLOUD==False')

We are sending an image url to a computer (in this case, one of Amazon's). Our api then takes that image url and passes it to our trained model and returns a prediction. 

### How to build an API in Python

#### Flask

Goals
- Explain the difference between a POST and GET requests
- Create an API on our local machine 
- Send a request to a specific "route" and have it return a response

GET Methods - GET is the default method in each route. GET Methods recieve a request and return some information. When you type something into your browsers search bar you sending a GET request. 

<img style='width:300px' src='images/get_rm.gif' /> 

Flask is a web application framework built in Python. It allows us to set up a server that is listening for a request and when it recieves the request it returns an appropriate response. 

In [ ]:
import flask

#initalize app
app = flask.Flask(__name__)

#when the route "/" recieves a request the function hello is run
@app.route("/predict")
def hello():
    return "Hello World!"

#run app
if __name__ == "__main__":
    app.run()

<img style='width:300px' src='images/get_rm.gif' /> 

#### POST Methods

In [ ]:
import flask

#initalize app
app = flask.Flask(__name__)

#when the route "/" recieves a request the function hello is run
@app.route("/")
def hello():
    return "Hello World!"
#'/predict' reutrns 'Request recieved' when it recieves a POST request
@app.route("/predict", methods=["POST", "GET"])
def pred():
    if flask.request.method == "POST":
        years = flask.request.form['years']
        return f'Request recieved {years}'
    else:
        return 'GET'
#run app
if __name__ == "__main__":
    app.run()

#### Get a prediction from a model

In [ ]:
import requests
response = requests.post('http://127.0.0.1:5000/predict')
response.content

In [ ]:
import pickle
import flask
import json
import numpy as np
#initalize app
app = flask.Flask(__name__)
#initialize model outside of route so it doesn't have to load everytime it recieves a request
model = pickle.load( open('model.pkl','rb'))

#when the route "/" recieves a request the function hello is run
@app.route("/")
def hello():
    return "Hello World!"


@app.route("/predict", methods=["POST"])
def pred():
    if flask.request.method == "POST":
        years = np.float(flask.request.form['years'])
        prediction = model.predict([[years]])
        data = {}
        data['predictions'] = prediction[0]
        return flask.jsonify(data)
#run app
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.3 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2020 12:56:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 12:56:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Nov/2020 12:57:27] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [03/Nov/2020 12:57

In [ ]:
import requests
response = requests.post('http://127.0.0.1:5000/predict', data={'years':1})
response.json()['predictions']

<b>Resources</b>

https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html <br/>
https://www.pyimagesearch.com/2018/01/29/scalable-keras-deep-learning-rest-api/

<h1 style='text-align:center'>Hosting Your API on AWS</h1>

Amazon offers services that allow us to use their computers to host our API. This allows other people to access our API at anytime from any computer connected to the internet. 

### Setting up AWS

https://aws.amazon.com/  - Set up an account and log in 

We will be using Elastic Cloud Compute aka EC2: 

<b>Steps</b>
1. Launch an instance (a virtual server) 
2. Connect to that instance via SSH (allows you to access the servers terminal from your computer) 
3. Upload your code onto the server and run it

### 1. Launch an Instance

<img src='images/select_ec2.png'/>

<b> Click "Launch Instance" Button</b>

1. Select which server OS you want to use. -  Ubuntu Server 18.04 is what I usually use. 
2. Select instance type (how many CPUs or GPUs) 
3. Click on Configure Security Group

<img src='images/security_groups.png'/>

4. Add rule of Type: HTTP, Port Range: 80, Source: Anywhere (SSH should be there by default)
3. Click "Review and Launch"
4. Launch
5. Create and download key pair - this is your "password" to log into your instance (don't lose this) 

<img src ='images/key_pair.png' width='600px'/>

### 2. Connect to Instance

1. Return to EC2 Console via Services -> EC2
2. Click on "Running Instances" 
3. Select the instance you created and click "Connect" 
4. Run the commands in your LOCAL terminal in the same directory that your {key_pair}.pem file is located


<img src='images/connect.png' width='600px'/>

<b>You're in! You can now interact with Amazon's server! </b>

<img src='images/terminal.png'/>

### 3. Upload Your Code and Run Your Server

You can git clone your repository into this server or you can upload single files via the terminal. Since we have a simple script we will upload the single file.

Our server code is pretty much ready to be deployed to AWS but we need to make one small change so when we go to our instances ip address we are routed to the correct port (80). The default HTTP port is 80 so our server needs to be there. 

In [ ]:
#replace app.run() in our api file with this
app.run(host='0.0.0.0', port=80)

In [ ]:
#run this in terminal to transfer your file to a specific folder in your server
#run in the same folder as your .pem file and the file you want to transfer
scp -i aws-lecture.pem api.py ubuntu@ec2-52-14-97-73.us-east-2.compute.amazonaws.com:~/

Before running our server we need install the neccesary dependencies. Python 3 is install by default on Ubunutu but we need to install pip first and then the other dependencies (flask, numpy, etc.) 

In [ ]:
#code to run in terminal
#install pip, dependencies and run server
sudo apt-get update
sudo apt install python3-pip
pip3 install Flask numpy sklearn
sudo python3 api.py

Now we can send requests to our server. To find our server's domain look for the "Public DNS (IPv4)"

<img src='images/ip.png'/>

In [ ]:
import requests
response = requests.post('http://ec2-52-14-97-73.us-east-2.compute.amazonaws.com/predict', data={'years':1})

In [ ]:
response.json()

In [ ]:
import requests
response = requests.post('http://127.0.0.1:5000/predict', data={'years':100})
response.content